In [1]:
import $ivy.`org.scala-lang:scala-library:2.13.12`

// Add Delta Lake dependency
import $ivy.`io.delta:delta-spark_2.13:3.1.0`

// Add Spark SQL dependency
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.1`
import $ivy.`org.apache.spark:spark-core_2.13:3.5.1`

//hadoop hdfs
import $ivy.`org.apache.hadoop:hadoop-common:3.3.6`
import $ivy.`org.apache.hadoop:hadoop-client:3.3.6`
import $ivy.`org.apache.hadoop:hadoop-client-api:3.3.6`
import $ivy.`org.apache.hadoop:hadoop-hdfs:3.3.6`

import $ivy.$                                     

// Add Delta Lake dependency

import $ivy.$                                

// Add Spark SQL dependency

import $ivy.$                                      

import $ivy.$                                       

//hadoop hdfs

import $ivy.$                                      

import $ivy.$                                      

import $ivy.$                                          

import $ivy.$                                    


In [ ]:
import io.delta.tables.DeltaTable
import org.apache.spark.sql.{Encoders, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [ ]:
val spark=SparkSession.builder()
    .appName("Fetch Delta Table")
    .master("local[*]")//runs on localhost 4040
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000")//setup for connecting to hadoop, comment it out if not required
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.databricks.delta.autoCompact.enabled", "true") // Enable auto-compaction globally. By default will use 128 MB as the target file size.
    .config("spark.databricks.delta.optimizeWrite.enabled", "true")
    .config("spark.databricks.delta.autoCompact.targetFileSize", "256MB") // Set target file size for auto-compaction
    .config("spark.databricks.delta.optimizeWrite.binSize", "256MB") // Set bin size for optimized writes
    .getOrCreate()

In [ ]:
val delPath="hdfs://localhost:9000/delta/dataF"
val delPath2="hdfs://localhost:9000/delta/dataFsms"
val delPath3="hdfs://localhost:9000/delta/dataFesewa"
val delPath4="/home/avyuthan-shah/Desktop/Data/dataF"

<h2>Z-Order</h2>

In [ ]:
writeFile("true")//to monitor load
val startTime = System.nanoTime()
val dt=DeltaTable.forPath(spark, delPath)
dt.optimize().executeZOrderBy("AccountNo")
val endTime = System.nanoTime()
writeFile("false")
val elapsedTime = (endTime - startTime) / 1e9 // Time in seconds
println(s"Elapsed time for Zordering: '$elapsedTime' ")

In [ ]:
val version=DeltaTable.forPath(spark, delPath).history(1).select("version").as[Long](Encoders.scalaLong).head //Get Latest Version
writeFile("true")//to monitor load
val startTime = System.nanoTime()
spark.read.format("delta").option("versionAsOf",version).load(delPath).createOrReplaceTempView("delta_table")

val result = spark.sql(
   s"""
   |SELECT * FROM delta_table
   |WHERE AccountNo IN ("56135","69310","65260")
   |ORDER BY AccountNo,VALUEDATE DESC,TIME DESC;
   """.stripMargin)
    
result.show(truncate=false)
val endTime = System.nanoTime()
writeFile("false")
val elapsedTime = (endTime - startTime) / 1e9 // Time in seconds
println(s"Elapsed time for retrival: '$elapsedTime' ")